# Exploring the Link between Safety Ratings and Car Price

### Marta Brasola 905305
##### September 2023

In [1]:
# import libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from itertools import zip_longest


## Scraping automoto.it

In [10]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://www.automoto.it/listino/ricerca'

driver.get(url)
driver.implicitly_wait(5)

banner_elements = driver.find_elements(By.ID,"iubenda-cs-banner")

for element in banner_elements:
    button = element.find_element(By.CSS_SELECTOR,"button.app-btn.app-btn-accept.iubenda-cs-accept-btn")
    button.click()
    
driver.implicitly_wait(10)

In [11]:
time.sleep(10)

container_element = driver.find_element(By.CSS_SELECTOR, "div.container-main")

pause_time = 1  

last_height = driver.execute_script("return arguments[0].scrollHeight;", container_element)

while True:
    driver.execute_script("arguments[0].scrollIntoView(false);", container_element)

    time.sleep(pause_time)

    new_height = driver.execute_script("return arguments[0].scrollHeight;", container_element)

    # Check if the scroll height no longer increases
    if new_height == last_height:
        break

    # Update
    last_height = new_height

In [12]:
soup = BeautifulSoup(driver.page_source, features="html.parser")

In [13]:
elements = soup.find_all("div", class_="col col-12 col-md-6 col-xl-4")

titles = []
prices = []
labels = []
ratings = []

for element in elements:
    
    # Extract title
    title_element = element.select("h2.plist-pcard-title.plist-pcard-title--mh a[title]")
    for title in title_element:
        if title:
            titles.append(title['title'])
        else:
            None

    # Extract price
    price_element = element.find("div", class_="plist-pcard-price")
    price = price_element.get_text(strip=True) if price_element else None
    if price is not None:
        prices.append(price)

    # Extract label
    label_element = element.find("div", class_="plist-pcard-label")
    label = label_element.get_text(strip=True) if label_element else None
    if label is not None:
        labels.append(label)


data = {
    "brand_model": titles,
    "price_range": prices,
    "setups": labels,

}
auto = pd.DataFrame(data)

In [14]:
#driver.quit()

In [15]:
auto.head()

,brand_model,price_range,setups
0,Abarth 500e,da €37.950\n a €43.000,3 allestimenti
1,Abarth 500e Cabrio,da €37.950\n a €43.000,3 allestimenti
2,Abarth 595,€26.800,1 allestimenti
3,Abarth 595 Cabrio,€28.800,1 allestimenti
4,Abarth 695,da €30.100\n a €31.700,2 allestimenti


In [16]:
auto.shape

(701, 3)

In [17]:
time.sleep(7)
driver.quit()
time.sleep(3)

## Cleaning the automoto.it dataset

In [18]:
regex_pattern = r'€([\d.]+)'
prices = auto['price_range'].str.findall(regex_pattern)

# If there are two prices in each row, assign them to 'min_price' and 'max_price'
auto['min_price'] = [p[0] if len(p) >= 1 else None for p in prices]
auto['max_price'] = [p[1] if len(p) == 2 else None for p in prices]

In [19]:
auto[auto['max_price'].isnull()]['setups'].unique()

array(['1 allestimenti', '4 allestimenti', '2 allestimenti'], dtype=object)

In [20]:
null = auto[auto['max_price'].isnull()]
null[(null['setups'] == '4 allestimenti') | (null['setups'] == '2 allestimenti')]

,brand_model,price_range,setups,min_price,max_price
287,Ineos Grenadier Pick-up,€67.410,4 allestimenti,67.410,None
614,Suzuki Across,€55.400,2 allestimenti,55.400,None
621,Tazzari Ev Minimax,€13.107,2 allestimenti,13.107,None


In [21]:
auto.head()

,brand_model,price_range,setups,min_price,max_price
0,Abarth 500e,da €37.950\n a €43.000,3 allestimenti,37.950,43.000
1,Abarth 500e Cabrio,da €37.950\n a €43.000,3 allestimenti,37.950,43.000
2,Abarth 595,€26.800,1 allestimenti,26.800,None
3,Abarth 595 Cabrio,€28.800,1 allestimenti,28.800,None
4,Abarth 695,da €30.100\n a €31.700,2 allestimenti,30.100,31.700


After double checking all cars with one single value and more than one setup I can confirm that for every single setup the price is the same.
Now I can correctly compute the None values of the max price with the same value of the minimum price.

In [22]:
auto['max_price'].fillna(auto['min_price'], inplace=True)

In [23]:
auto.isnull().sum()

brand_model    0
price_range    0
setups         0
min_price      0
max_price      0
dtype: int64

In [24]:
auto['min_price'] = auto['min_price'].str.replace('.', '').astype(int)
auto['max_price'] = auto['max_price'].str.replace('.', '').astype(int)

auto.drop(columns='price_range', inplace=True)

In [25]:
auto.head()

,brand_model,setups,min_price,max_price
0,Abarth 500e,3 allestimenti,37950,43000
1,Abarth 500e Cabrio,3 allestimenti,37950,43000
2,Abarth 595,1 allestimenti,26800,26800
3,Abarth 595 Cabrio,1 allestimenti,28800,28800
4,Abarth 695,2 allestimenti,30100,31700


In [26]:
# Split the column into "brand" and "model"
auto['brand'] = auto['brand_model'].str.split(n=1).str[0]
auto['model'] = auto['brand_model'].str.split(n=1).str[1]

# Handle the special case for "Alfa Romeo"
mask1 = auto['brand_model'].str.startswith('Alfa Romeo')
auto.loc[mask1, 'brand'] = 'Alfa Romeo'
auto.loc[mask1, 'model'] = auto.loc[mask1, 'brand_model'].str.split(n=2).str[2]

# Handle the special case for "Alfa Romeo"
mask2 = auto['brand_model'].str.startswith('Aston Martin')
auto.loc[mask2, 'brand'] = 'Aston Martin'
auto.loc[mask2, 'model'] = auto.loc[mask2, 'brand_model'].str.split(n=2).str[2]

In [27]:
auto['brand_model'] = auto['brand_model'].str.lower()
auto['brand'] = auto['brand'].str.lower()
auto['model'] = auto['model'].str.lower()

In [28]:
auto.head()

,brand_model,setups,min_price,max_price,brand,model
0,abarth 500e,3 allestimenti,37950,43000,abarth,500e
1,abarth 500e cabrio,3 allestimenti,37950,43000,abarth,500e cabrio
2,abarth 595,1 allestimenti,26800,26800,abarth,595
3,abarth 595 cabrio,1 allestimenti,28800,28800,abarth,595 cabrio
4,abarth 695,2 allestimenti,30100,31700,abarth,695


In [89]:
auto = auto.drop(auto[auto['model'].str.contains('furgone')].index).reset_index(drop=True)

In [90]:
auto = auto.drop(auto[auto['model'].str.contains('telaio')].index).reset_index(drop=True)

In [91]:
auto.shape

(625, 6)

## Sraping Euro NCAP Website

In [52]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://www.euroncap.com/en/ratings-rewards/latest-safety-ratings/#?selectedMake=0&selectedMakeName=Select%20a%20make&selectedModel=0&selectedStar=&includeFullSafetyPackage=true&includeStandardSafetyPackage=true&selectedModelName=All&selectedProtocols=49446,45155,41776,40302,34803,30636,26061,24370,-1&selectedClasses=1202,1199,1201,1196,1205,1203,1198,1179,40250,1197,1204,1180,34736,44997&allClasses=true&allProtocols=true&allDriverAssistanceTechnologies=false&selectedDriverAssistanceTechnologies=&thirdRowFitment=false"
driver.get(url)
driver.implicitly_wait(5)

button = driver.find_element(By.ID, "impliedsubmit")
button.click()

time.sleep(10)

In [53]:
soup = BeautifulSoup(driver.page_source, features="html.parser")

In [55]:
elements = soup.find_all(id="rating-2014")

In [57]:
data = []

for element in elements:
    rating_year = element.find("h3", class_="ng-binding").get_text(strip=True)
    
    sub = element.select("div.rating-table-row.ng-scope")
    
    for row in sub:
        model_elements = row.find_all(class_="name ng-binding")
        
        safety_equip_elements = row.select("div.rating-table-row-c.c10")

        star_elements = row.find_all(class_="stars")
        
        adult_elements = row.select("div.rating-table-row-c.c3.note.adult-occupant") 
        child_elements = row.select("div.rating-table-row-c.c4.note.child-occupant") 
        pedestrian_elements = row.select("div.rating-table-row-c.c5.note.pedestrian")
        safety_elements = row.select("div.rating-table-row-c.c6.note.safety-assist")
        
        for model_element, safety_equip_element, star_element, adult_element, child_element, pedestrian_element, safety_element in zip_longest(model_elements, safety_equip_elements,
                                                                                                                                       star_elements, adult_elements,child_elements,
                                                                                                                                      pedestrian_elements,safety_elements, fillvalue=None):
            model_name = model_element.get_text(strip=True)
            
            safety_equip = safety_equip_element.get_text(strip=True) if safety_equip_element is not None else None
            
            star_rating = star_element.find("img").get("alt")

            perc1 = adult_element.get_text(strip=True)
            perc2 = child_element.get_text(strip=True)
            perc3 = pedestrian_element.get_text(strip=True)
            perc4 = safety_element.get_text(strip=True)

            data.append({"brand_model": model_name, 
                         "safety_equipement": safety_equip,  
                         "rating_year": rating_year, 
                         "star_rating": star_rating,
                         'adult_occupant_safety': perc1,
                         'child_occupant_safety': perc2,
                         'road_users_safety': perc3,
                         'safety_assist': perc4})

euro = pd.DataFrame(data)


In [58]:
euro.head()

,brand_model,safety_equipement,rating_year,star_rating,adult_occupant_safety,child_occupant_safety,road_users_safety,safety_assist
0,Lexus RZ,Standard,2023 - Rating,5,87%,87%,84%,81%
1,NIO EL7,Standard,2023 - Rating,5,93%,85%,80%,79%
2,NIO ET5,Standard,2023 - Rating,5,96%,85%,83%,81%
3,Jeep Grand Cherokee,Standard,2022 - Rating,5,84%,89%,81%,81%
4,CHERY OMODA5,Standard,2022 - Rating,5,87%,87%,68%,88%


In [59]:
euro.shape

(587, 8)

In [60]:
time.sleep(7)
driver.quit()

# Cleaning the Euro NCAP dataset

In [61]:
euro['rating_year'] = euro['rating_year'].str.split(n=1).str[0]

In [62]:
euro['adult_occupant_safety'] = euro['adult_occupant_safety'].str.replace('%','').astype(int)
euro['child_occupant_safety'] = euro['child_occupant_safety'].str.replace('%','').astype(int)
euro['road_users_safety'] = euro['road_users_safety'].str.replace('%','').astype(int)
euro['safety_assist'] = euro['safety_assist'].str.replace('%','').astype(int)

In [63]:
euro['brand'] = euro['brand_model'].str.split(n=1).str[0]
euro['model'] = euro['brand_model'].str.split(n=1).str[1]

mask = euro['brand_model'].str.startswith('Alfa Romeo')
euro.loc[mask, 'brand'] = 'Alfa Romeo'
euro.loc[mask, 'model'] = euro.loc[mask, 'brand_model'].str.split(n=2).str[2]

In [64]:
euro['brand_model'] = euro['brand_model'].str.lower()
euro['brand'] = euro['brand'].str.lower()
euro['model'] = euro['model'].str.lower()

In [65]:
euro[euro['brand']=='alfa romeo'][['brand_model','brand','model']]

,brand_model,brand,model
50,alfa romeo tonale,alfa romeo,tonale
212,alfa romeo giulietta,alfa romeo,giulietta
258,alfa romeo stelvio,alfa romeo,stelvio
299,alfa romeo giulia,alfa romeo,giulia
532,alfa romeo giulietta,alfa romeo,giulietta


In [66]:
euro.head()

,brand_model,safety_equipement,rating_year,star_rating,adult_occupant_safety,child_occupant_safety,road_users_safety,safety_assist,brand,model
0,lexus rz,Standard,2023,5,87,87,84,81,lexus,rz
1,nio el7,Standard,2023,5,93,85,80,79,nio,el7
2,nio et5,Standard,2023,5,96,85,83,81,nio,et5
3,jeep grand cherokee,Standard,2022,5,84,89,81,81,jeep,grand cherokee
4,chery omoda5,Standard,2022,5,87,87,68,88,chery,omoda5


In [67]:
euro.shape

(587, 10)

In [68]:
euro['safety_equipement'] = euro['safety_equipement'].fillna('no_info')

In [69]:
euro[euro['brand_model']=='mazda 6']

,brand_model,safety_equipement,rating_year,star_rating,adult_occupant_safety,child_occupant_safety,road_users_safety,safety_assist,brand,model
201,mazda 6,Standard,2018,5,95,91,66,73,mazda,6
408,mazda 6,no_info,2013,5,92,77,66,81,mazda,6
580,mazda 6,no_info,2009,5,77,81,49,71,mazda,6


In [70]:
euro[euro['brand_model']=='kia niro']

,brand_model,safety_equipement,rating_year,star_rating,adult_occupant_safety,child_occupant_safety,road_users_safety,safety_assist,brand,model
43,kia niro,Standard,2022,4,91,84,75,60,kia,niro
44,kia niro,Safety Pack,2022,5,91,84,76,79,kia,niro
293,kia niro,Standard,2016,4,83,80,57,59,kia,niro
294,kia niro,Safety Pack,2016,5,91,80,70,81,kia,niro


In [71]:
euro = euro.drop(euro[euro['safety_equipement'].str.contains('Safety Pack')].index).reset_index(drop=True)

In [72]:
euro.shape

(570, 10)

In [73]:
euro.drop_duplicates(subset='brand_model', inplace=True, ignore_index=True)

In [77]:
euro.shape

(450, 10)

## Merging the datasets

In [93]:
auto.shape

(625, 6)

In [94]:
merge = auto.merge(euro, on=['brand','model','brand_model'], how='left')

In [95]:
merge.shape

(625, 13)

### Uploading everything to Mongo 

In [100]:
import pymongo 
import json

In [97]:
df = merge[merge['rating_year'].notnull()].reset_index(drop=True)

In [98]:
df.shape

(169, 13)

In [101]:
client = pymongo.MongoClient("mongodb://localhost:27017")

In [102]:
data = df.to_dict(orient = "records")

In [103]:
db = client["ProjectDataManagement"]

In [104]:
db.Cars.insert_many(data)